In [ ]:
# Importar Bibliotecas Necessárias
import pandas as pd
import requests
import mlflow
from sklearn.metrics import log_loss, f1_score

# Definir o nome da rodada do MLflow
mlflow.start_run(run_name="PipelineAplicacao")

# Carregar a base de produção
prod_data_path = 'Proj_Kobe-Bryant-Shot-Selection/data/raw/dataset_kobe_prod.parquet'
df_prod = pd.read_parquet(prod_data_path)

# Carregar o modelo registrado no MLflow
model_path = "model"  # Caminho para o modelo no MLflow (deve ser ajustado conforme necessário)
model = mlflow.sklearn.load_model(model_path)

# Fazer previsões na base de produção
X_prod = df_prod.drop(columns=['shot_made_flag']) 
y_prod_pred = model.predict(X_prod)
y_prod_pred_proba = model.predict_proba(X_prod)[:, 1]

# Calcular métricas na base de produção
y_prod_true = df_prod['shot_made_flag']
log_loss_prod = log_loss(y_prod_true, y_prod_pred_proba)
f1_score_prod = f1_score(y_prod_true, y_prod_pred)

# Salvar resultados em uma tabela e como artefato .parquet
results_df = pd.DataFrame({'shot_made_flag': y_prod_true, 'predicted_proba': y_prod_pred_proba})
results_df.to_parquet('Proj_Kobe-Bryant-Shot-Selection/data/processed/predictions_results.parquet', index=False)

# Log das métricas
mlflow.log_metric("log_loss_prod", log_loss_prod)
mlflow.log_metric("f1_score_prod", f1_score_prod)

# Finalizar a rodada do MLflow
mlflow.end_run()

# Implementação da API local
def predict_api(input_data):
    """
    Função para fazer previsões usando a API local.
    """
    # URL da API
    url = 'http://127.0.0.1:5000/predict'
    
    # Fazer a solicitação POST para a API
    response = requests.post(url, json=input_data)
    
    # Retornar a resposta da API
    return response.json()

# Exemplo de uso da API
input_data_example = {"lat": 34.0443, "lon": -118.2698, "minutes_remaining": 10, "period": 1, 
                      "playoffs": 0, "shot_distance": 15}
prediction = predict_api(input_data_example)
print("Exemplo de previsão usando a API local:", prediction)
